In [8]:
%run DataProcessing.ipynb

2024-08-14 04:27:39.483113
2004-08-14 04:27:39.483113


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


           Date  ^GSPC_change  ^GSPC_lag
0    2004-08-16     14.539917        NaN
1    2004-08-17      2.369995  14.539917
2    2004-08-18     13.460083   2.369995
3    2004-08-19     -3.940063  13.460083
4    2004-08-20      7.119995  -3.940063
...         ...           ...        ...
5028 2024-08-07    -40.560059  86.709961
5029 2024-08-08     62.090332 -40.560059
5030 2024-08-09     37.219727  62.090332
5031 2024-08-12     25.100098  37.219727
5032 2024-08-13           NaN  25.100098

[5033 rows x 3 columns]
           Date  ^GSPC_change  ^GSPC_lag       ^IXIC        ^DJI      ^FCHI  \
0    2004-08-16     14.539917        NaN         NaN         NaN        NaN   
1    2004-08-17      2.369995  14.539917   32.680054  130.150391  42.409912   
2    2004-08-18     13.460083   2.369995   -4.599976    8.719727  15.120117   
3    2004-08-19     -3.940063  13.460083   38.250000  118.560547  29.449951   
4    2004-08-20      7.119995  -3.940063   -6.369995  -41.970703 -11.910156   
...      

In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset

### Create DataLoader instances

In [10]:
train_dataset = TensorDataset(X_train_seq, y_train_seq)
test_dataset = TensorDataset(X_test_seq, y_test_seq)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)